<a href="https://colab.research.google.com/github/sadaha/Coursera_Capstone.-/blob/main/IBM_Capstone_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IBM Capstone Project**

## Here I will write my report, and code for the IBM capstone project.

Problem Description:

> The city used for the project will be Hamburg. The problem will be looking at types of restaurant in the city, such as Italian, German, Korean, Chinese etc, and trying to cluster the different neighbourhoods by which type of restaunts are most populous in which areas, such that the client, a new restauranteer who wants to open a Korean restaurant, can see areas of potentially high and low competition.



Data Description:

> The data will be the locations and types of restaurants in each city borough, as well as their average rating. To be used to determine how much competition they could provide. The data will be used to look at areas of high and low restaurant density (of all types of cuisine), as well as look at the frequency of korean restaurants present in each city sector. The data will be sourced from Foursquare if I can ever get the site to let me make an account. Else I will find a CSV dataset somewhere.

# Obtaining the Data

Hamburg Areas:

> Here I will scrape the different locations in Hamburg from Wikipedia by quarter, and the postcodes for each area. The area information we will take from the following site https://www.citypopulation.de/en/germany/hamburg/admin/

From this we will take the name of the quarter and the population as most recently measured.

In [ ]:
hamburg_df_raw = hamburg_df_raw[0]
hamburg_df_raw

,Name,Status,PopulationEstimate2002-12-31,PopulationEstimate2007-12-31,PopulationEstimate2012-12-31,PopulationEstimate2017-12-31,PopulationEstimate2019-12-31,Unnamed: 7
0,Altona,Borough,...,257299,257412,270263,275265,→
1,Altona-Altstadt,Quarter,27550,26918,28174,29034,29170,→
2,Altona-Nord,Quarter,21662,22521,21759,22137,24536,→
3,Bahrenfeld,Quarter,26343,27193,27165,31047,30203,→
4,Blankenese,Quarter,13452,13435,13016,13407,13730,→
...,...,...,...,...,...,...,...,...
108,Volksdorf,Quarter,18818,19362,20115,20625,20978,→
109,Wandsbek,Quarter,32583,33034,33131,34469,36444,→
110,Wellingsbüttel,Quarter,9432,9829,10263,10506,10848,→
111,Wohldorf-Ohlstedt,Quarter,4189,4405,4423,4656,4650,→


We only need the "Name", "State" and last Population collumn, so remove the rest.

In [ ]:
hamburg_df = hamburg_df_raw.drop(['PopulationEstimate2002-12-31', 'PopulationEstimate2007-12-31','PopulationEstimate2012-12-31', 'PopulationEstimate2017-12-31', 'Unnamed: 7'], axis=1)
hamburg_df

,Name,Status,PopulationEstimate2019-12-31
0,Altona,Borough,275265
1,Altona-Altstadt,Quarter,29170
2,Altona-Nord,Quarter,24536
3,Bahrenfeld,Quarter,30203
4,Blankenese,Quarter,13730
...,...,...,...
108,Volksdorf,Quarter,20978
109,Wandsbek,Quarter,36444
110,Wellingsbüttel,Quarter,10848
111,Wohldorf-Ohlstedt,Quarter,4650


Curently the data frame has Quarters and Boroughs, we want to remove the Borough values

In [ ]:
indexNames = hamburg_df[ hamburg_df['Status'] == 'Borough' ].index
hamburg_df.drop(indexNames , inplace=True)
hamburg_df

,Name,Status,PopulationEstimate2019-12-31
1,Altona-Altstadt,Quarter,29170
2,Altona-Nord,Quarter,24536
3,Bahrenfeld,Quarter,30203
4,Blankenese,Quarter,13730
5,Groß Flottbek,Quarter,11111
...,...,...,...
108,Volksdorf,Quarter,20978
109,Wandsbek,Quarter,36444
110,Wellingsbüttel,Quarter,10848
111,Wohldorf-Ohlstedt,Quarter,4650


The last value is the city itself, so remove that too

In [ ]:
hamburg_df.drop(hamburg_df.tail(1).index,inplace=True)
hamburg_df

,Name,Status,PopulationEstimate2019-12-31
1,Altona-Altstadt,Quarter,29170
2,Altona-Nord,Quarter,24536
3,Bahrenfeld,Quarter,30203
4,Blankenese,Quarter,13730
5,Groß Flottbek,Quarter,11111
...,...,...,...
107,Tonndorf,Quarter,15148
108,Volksdorf,Quarter,20978
109,Wandsbek,Quarter,36444
110,Wellingsbüttel,Quarter,10848


Now we can find the latitude and longitude data for each Quarter and add them to the dataframe. To start we install arcgid.

Separate the location names into a seperate dataframe

In [ ]:
geo_coordinates = hamburg_df['Name']  
geo_coordinates

1        Altona-Altstadt
2            Altona-Nord
3             Bahrenfeld
4             Blankenese
5          Groß Flottbek
             ...        
107             Tonndorf
108            Volksdorf
109             Wandsbek
110       Wellingsbüttel
111    Wohldorf-Ohlstedt
Name: Name, Length: 105, dtype: object

Get the Latitude and Longitude data for each area in hamburg

In [ ]:
coordinates_latlong = geo_coordinates.apply(lambda x: get_x_y(x))
coordinates_latlong

1        53.55078000000003,9.940710000000024
2        53.56804000000005,9.948140000000024
3         53.56466000000006,9.90166000000005
4        53.56314000000003,9.812320000000057
5        53.56564000000003,9.877840000000049
                       ...                  
107    53.586970000000065,10.127130000000022
108     53.65131000000008,10.167030000000068
109     53.57205000000005,10.067070000000058
110    53.640640000000076,10.083020000000033
111     53.69557000000003,10.139000000000067
Name: Name, Length: 105, dtype: object

Make a single dataframe with all the data

In [ ]:
Hamburg_df_complete = pd.concat([hamburg_df,lat_Hamburg.astype(float), lon_Hamburg.astype(float)], axis=1)
Hamburg_df_complete.columns= ['Name','Status','PopulationEstimate2019-12-31','latitude','longitude']
Hamburg_df_complete

,Name,Status,PopulationEstimate2019-12-31,latitude,longitude
1,Altona-Altstadt,Quarter,29170,53.55078,9.94071
2,Altona-Nord,Quarter,24536,53.56804,9.94814
3,Bahrenfeld,Quarter,30203,53.56466,9.90166
4,Blankenese,Quarter,13730,53.56314,9.81232
5,Groß Flottbek,Quarter,11111,53.56564,9.87784
...,...,...,...,...,...
107,Tonndorf,Quarter,15148,53.58697,10.12713
108,Volksdorf,Quarter,20978,53.65131,10.16703
109,Wandsbek,Quarter,36444,53.57205,10.06707
110,Wellingsbüttel,Quarter,10848,53.64064,10.08302


# Finding restaurants in Hamburg

Now to use Foursquare (if it works) to find the restaurant locations in hamburg

Foursquare doesn't want to play nicely so I have found a CSV file from https://www.europeandataportal.eu/data/datasets/32d2e295-08c0-446e-8bf8-5a7d3b315d51?locale=de containing a large amount of restaurant data to use.

Import the CSV file

In [ ]:
Restuarant_data = pd.read_csv('Tabelle1.csv')
Restuarant_data

,Restaurant,Adresse,Beschreibung,URL mit weiteren Informationen
0,Abendmahl,"Hein-Köllisch-Platz 6, 20359 Hamburg",Abseits vom Trubel auf dem Kiez bietet das Abe...,http://www.hamburg-tourism.de/erleben/essen-tr...
1,Al Volo,"Eppendorfer Weg 211, 20253 Hamburg",Authentisch italienische Küche mit hervorragen...,http://www.hamburg-tourism.de/erleben/essen-tr...
2,Alsterperle,"Eduard-Rhein-Ufer 1, 22087 Hamburg",Süßes Lokal am Ost-Zipfel des Alstersees,http://www.hamburg-tourism.de/erleben/essen-tr...
3,Alt Hamburger Aalspeicher,"Deichstraße 43, 20459 Hamburg",In einem historischen Gebäude aus dem 16. Jahr...,http://www.hamburg-tourism.de/erleben/essen-tr...
4,An Khang Quan,"Hoheluftchaussee 86, 20253 Hamburg",Einer der meist besuchten Vietnamesen in Hambu...,http://www.hamburg-tourism.de/erleben/essen-tr...
...,...,...,...,...
149,Williamine,"Kleiner Schäferkamp 16, 20357 Hamburg",Wenn Essen zum Erlebnis wird...,http://www.hamburg-tourism.de/erleben/essen-tr...
150,Zipang,"Eppendorfer Weg 171, 20253 Hamburg",Im Zipang fusionieren die authentisch japanisc...,http://www.hamburg-tourism.de/erleben/essen-tr...
151,Zum Alten Lotsenhaus,"Övelgönne 13, 22605 Hamburg",In den geschichtsträchtigen Wänden des Alten L...,http://www.hamburg-tourism.de/erleben/essen-tr...
152,Zum Schiffchen,"Großer Grasbrook 9, 20457 Hamburg",Das im Brauhausstil gehaltene Restaurant ist e...,http://www.hamburg-tourism.de/erleben/essen-tr...


Here we have the restuarant names, and locations. Unforunately the type of restuarant is hard to ascertain. Maybe would be possible looking through the descriptive text, but likely not for each example.

Let's remove the last two columns. And find the latitude and longitude data for each restaurant.

In [ ]:
Restuarant_data_reduced = Restuarant_data.drop(['Beschreibung', 'URL mit weiteren Informationen'], axis=1)
Restuarant_data_reduced

,Restaurant,Adresse
0,Abendmahl,"Hein-Köllisch-Platz 6, 20359 Hamburg"
1,Al Volo,"Eppendorfer Weg 211, 20253 Hamburg"
2,Alsterperle,"Eduard-Rhein-Ufer 1, 22087 Hamburg"
3,Alt Hamburger Aalspeicher,"Deichstraße 43, 20459 Hamburg"
4,An Khang Quan,"Hoheluftchaussee 86, 20253 Hamburg"
...,...,...
149,Williamine,"Kleiner Schäferkamp 16, 20357 Hamburg"
150,Zipang,"Eppendorfer Weg 171, 20253 Hamburg"
151,Zum Alten Lotsenhaus,"Övelgönne 13, 22605 Hamburg"
152,Zum Schiffchen,"Großer Grasbrook 9, 20457 Hamburg"


Now to find the location data for each restuarant using geocode.

In [ ]:
geo_coordinates_rest = Restuarant_data_reduced['Adresse']  
geo_coordinates_rest

0      Hein-Köllisch-Platz  6, 20359 Hamburg
1        Eppendorfer Weg 211, 20253 Hamburg 
2         Eduard-Rhein-Ufer 1, 22087 Hamburg
3              Deichstraße 43, 20459 Hamburg
4         Hoheluftchaussee 86, 20253 Hamburg
                       ...                  
149    Kleiner Schäferkamp 16, 20357 Hamburg
150       Eppendorfer Weg 171, 20253 Hamburg
151              Övelgönne 13, 22605 Hamburg
152        Großer Grasbrook 9, 20457 Hamburg
153     Karl-Theodor-Straße 4, 22765 Hamburg
Name: Adresse, Length: 154, dtype: object

In [ ]:
coordinates_latlong_rest = geo_coordinates_rest.apply(lambda x: get_x_y(x))
coordinates_latlong_rest

0       53.54759699364547,9.956741244512024
1       53.581406741208134,9.97058898674815
2      53.56799448075142,10.015722936957875
3      53.545488819268286,9.986939481453483
4       53.58359004510686,9.970378643016758
                       ...                 
149     53.56711647639385,9.968927442081673
150     53.579586569024094,9.96648830825763
151     53.54483746157253,9.912018175195898
152    53.541787915649735,9.993654307906377
153    53.548571515618164,9.931221199189865
Name: Adresse, Length: 154, dtype: object

In [ ]:
Hamburg_df_rest_complete = pd.concat([Restuarant_data_reduced,lat_Hamburg_rest.astype(float), lon_Hamburg_rest.astype(float)], axis=1)
Hamburg_df_rest_complete.columns= ['Restaurant','Adresse','latitude','longitude']
Hamburg_df_rest_complete

,Restaurant,Adresse,latitude,longitude
0,Abendmahl,"Hein-Köllisch-Platz 6, 20359 Hamburg",53.547597,9.956741
1,Al Volo,"Eppendorfer Weg 211, 20253 Hamburg",53.581407,9.970589
2,Alsterperle,"Eduard-Rhein-Ufer 1, 22087 Hamburg",53.567994,10.015723
3,Alt Hamburger Aalspeicher,"Deichstraße 43, 20459 Hamburg",53.545489,9.986939
4,An Khang Quan,"Hoheluftchaussee 86, 20253 Hamburg",53.583590,9.970379
...,...,...,...,...
149,Williamine,"Kleiner Schäferkamp 16, 20357 Hamburg",53.567116,9.968927
150,Zipang,"Eppendorfer Weg 171, 20253 Hamburg",53.579587,9.966488
151,Zum Alten Lotsenhaus,"Övelgönne 13, 22605 Hamburg",53.544837,9.912018
152,Zum Schiffchen,"Großer Grasbrook 9, 20457 Hamburg",53.541788,9.993654


Plot the restaurants on a map to visualise.

In [ ]:
# adding markers to map
for latitude, longitude, restaurant in zip(Hamburg_df_rest_complete['latitude'], Hamburg_df_rest_complete['longitude'], Hamburg_df_rest_complete['Restaurant']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True
        ).add_to(map)  
    
map

Now we have a map with the boroughts in red, and the restaurants in blue. And we can alread begin to see some clusters.

# Modelling

Now we can begin the modelling. I will use euclidian distances based off of the positional data to cluster the model. I will aim to make 5-6 clusters.

In [ ]:
Hamburg_df_rest_complete.insert(0, 'Cluster Labels', kmeans_Hamburg.labels_ +1)
Hamburg_df_rest_complete

,Cluster Labels,Restaurant,Adresse,latitude,longitude
0,7,Abendmahl,"Hein-Köllisch-Platz 6, 20359 Hamburg",53.547597,9.956741
1,1,Al Volo,"Eppendorfer Weg 211, 20253 Hamburg",53.581407,9.970589
2,5,Alsterperle,"Eduard-Rhein-Ufer 1, 22087 Hamburg",53.567994,10.015723
3,3,Alt Hamburger Aalspeicher,"Deichstraße 43, 20459 Hamburg",53.545489,9.986939
4,1,An Khang Quan,"Hoheluftchaussee 86, 20253 Hamburg",53.583590,9.970379
...,...,...,...,...,...
149,1,Williamine,"Kleiner Schäferkamp 16, 20357 Hamburg",53.567116,9.968927
150,1,Zipang,"Eppendorfer Weg 171, 20253 Hamburg",53.579587,9.966488
151,4,Zum Alten Lotsenhaus,"Övelgönne 13, 22605 Hamburg",53.544837,9.912018
152,3,Zum Schiffchen,"Großer Grasbrook 9, 20457 Hamburg",53.541788,9.993654


In [ ]:
map_clusters_hamburg = folium.Map(location=[Hamburg_lat_coords, Hamburg_lng_coords], zoom_start=12)


# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Hamburg_df_rest_complete['latitude'], Hamburg_df_rest_complete['longitude'], Hamburg_df_rest_complete['Restaurant'], Hamburg_df_rest_complete['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_hamburg)
        
map_clusters_hamburg

# Conclusion and results

From the data is is apparent that there is a lower density of restaurants in the areas of eppendorf, and in Rotherbaum.
As such the recommendation for the client would be to open their new restaurant in one of these locations.
There are two small clusters in Blankenese and Neinstedten, but being so far from the city they are likely not good choices for restuarant locations.

For the maps with the clusters visualised please see the slides attatched.


# Conclusion

I would conclude the work by advising the client to not open a restaruant as we're in the middle of a pandemic and I don't think it'd be a good idea right now.